In [139]:
import numpy as np
import time

In [140]:
def angular_dist(r1, d1, r2, d2):
  b = np.cos(d1)*np.cos(d2)*(np.sin(np.abs(r1 - r2)/2)**2)
  a = np.sin(np.abs(d1 - d2)/2)**2
  return 2*np.arcsin(np.sqrt(a + b))

In [141]:
def hms2dec(hr, m, s):
  dec = hr + m/60 + s/3600
  return dec*15

def dms2dec(d, m, s):
  sign = d/abs(d)
  dec = abs(d) + m/60 + s/3600
  return sign*dec

In [142]:
def import_bss():
  res = []
  data = np.loadtxt('table2.dat', usecols=range(1, 7))
  for datum in data:
    res.append([hms2dec(datum[0], datum[1], datum[2]), dms2dec(datum[3], datum[4], datum[5])])
  return np.array(res)

In [143]:
def find_closest(cat, r, d):
    min_dist = np.inf
    min_id = None
    for i, row in enumerate(cat):
        ang_dist = angular_dist(row[0], row[1], r, d)
        if ang_dist < min_dist:
            min_dist = ang_dist
            min_id = i
    return np.array([min_id, min_dist])

In [144]:
def crossmatch(cat1, cat2, max_dist):
    cat1 = np.radians(cat1)
    cat2 = np.radians(cat2)
    matches = []
    no_matches = []

    for i, row in enumerate(cat1):
        closest = find_closest(cat2, row[0], row[1])
        if closest[1] <= max_dist :
            matches.append((i, closest[0], closest[1]))
        else:
            no_matches.append(i)
    return matches, no_matches


In [145]:
def generate_cat(n):
    ra = np.random.uniform(0, 360, size=(n, 1))
    de = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ra, de))

In [146]:
def angular_dist_rad(r1, d1, r2, d2):
    deltar = np.abs(r1 - r2)
    deltad = np.abs(d1 - d2)
    angle = 2*np.arcsin(np.sqrt(np.sin(deltad/2)**2 
                        + np.cos(d1)*np.cos(d2)*np.sin(deltar/2)**2))
    return angle

def crossmatch_box(coords1, coords2):
    start_time = time.perf_counter()
    deg2rad = np.pi/180
    rad2deg = 180/np.pi
    max_radius = 5*deg2rad
    matches = []
    no_matches = []
    coords1_len = len(coords1)
    coords2_len = len(coords2)
    
    # Convert coordinates to radians
    coords1 = np.radians(coords1)
    coords2 = np.radians(coords2)
    
    # Find ascending declination order of second catalogue
    asc_dec = np.argsort(coords2[:, 1])
    coords2_sorted = coords2[asc_dec]
    dec2_sorted = coords2_sorted[:, 1]
    
    for id1 , (ra1, dec1) in enumerate(coords1):
        closest_dist = np.inf
        closest_id2 = None
        
        # Declination search box
        min_dec = dec1 - max_radius
        max_dec = dec1 + max_radius
        
        # Start and end indices of the search
        start = dec2_sorted.searchsorted(min_dec, side='left')
        end = dec2_sorted.searchsorted(max_dec, side='right')
        
        for s_id2, (ra2, dec2) in enumerate(coords2_sorted[start:end+1]):
            dist = angular_dist_rad(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_sorted_id2 = s_id2
                closest_dist = dist
        
        # Ignore match if it's outside the maximum radius
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            closest_id2 = asc_dec[closest_sorted_id2]
            matches.append([id1, closest_id2, closest_dist*rad2deg])
    
    time_taken = time.perf_counter() - start_time
    return matches, no_matches, time_taken

In [147]:
bss = import_bss()
cat2 = generate_cat(100)
box = crossmatch_box(bss, cat2)
start = time.perf_counter()
matches, no_matches = crossmatch(bss, cat2, 5/3600)
time_taken = time.perf_counter() - start


In [148]:
box[2], time_taken

(0.07207999300044321, 0.8352901380003459)

In [153]:
from astropy.coordinates import SkyCoord
from astropy import units as u
start = time.perf_counter()
sky_cat1 = SkyCoord(bss*u.degree, frame='icrs')
sky_cat2 = SkyCoord(cat2*u.degree, frame='icrs')
closest_ids, closest_dists, closest_dists3d = sky_cat1.match_to_catalog_sky(sky_cat2)
time_taken = time.perf_counter() - start
print(time_taken,"\n", (closest_ids, closest_dists.value, closest_dists3d))

0.01565513000059582 
 (array([14, 66, 66, 14, 66, 14, 53, 23, 14, 23, 10, 10, 14, 76, 76, 76, 14,
       76, 14, 61, 61, 76, 61, 76, 61, 76, 14, 61, 61, 10, 61, 76, 36, 76,
       61, 36, 36, 36, 10, 76, 76, 61, 61, 16, 18, 16, 18, 18, 61, 61, 79,
       71, 18, 65, 65, 18, 79, 65, 65, 71, 71, 79, 65, 79, 28, 79, 65, 28,
       65, 28, 65, 79, 71, 65, 65, 79, 65, 65, 65, 84, 65, 79, 79, 84, 84,
       28, 84, 79, 84, 84, 84, 28, 84, 84, 84, 84,  6, 84, 28, 28,  6,  6,
        6,  6,  0,  6,  0,  6, 28,  6,  6,  0,  6, 62,  6,  0, 62,  0, 57,
       57, 57, 62,  0,  0,  0, 57,  0, 57, 75,  9,  0, 57,  0,  0, 75,  9,
        0, 47, 75,  0,  0, 68,  0, 75, 72, 72, 68, 68, 40, 75, 75, 68, 72,
       75, 47, 72, 68, 68, 68, 68, 72, 72, 68, 68, 68, 72, 75, 68, 89,  3,
       68, 68, 45, 72, 45,  3,  3,  3,  3, 45, 45,  3, 45,  3, 45, 24,  3,
       83,  1,  1, 45, 83, 83, 24, 24, 83, 83, 83, 24,  1, 24,  1, 24, 42,
        1, 42, 93, 29, 47,  7, 89,  7, 93, 93, 93, 29, 29, 89, 29,  7,  7,
  